In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

## Code which scrapes the recently reviews (only 100 pages - 1,400 reviews)

In [2]:

def ratebeer_scraper(page_start=1, page_end=2, url_set=None):
    
    beer_name = []
    beer_rating = []
    brewery_name = []
    brewery_location = []
    review_content = []
    reviewer_name = []
 
    if url_set == None:
        print("Need url to run")
        return

    for x in tqdm(range(page_start, page_end + 1)):
        try:
            url_format = url_set.format(x)
            r = requests.get(url_format)
            soup = BeautifulSoup(r.text, 'html.parser')
            table = soup.find('table', attrs={'class': 'table'})
            table_rows = table.find_all('tr')

            for rows in table_rows:

            # Scrape Beer Name    
                try:
                    beer_name.append(rows.find('a', attrs={'style':'font-size:20px; font-weight:bold;'}).text)
                except:
                    beer_name.append(None)
            # Scrape Review Rating  
                try:
                    beer_rating.append(rows.find('span', attrs={'class':'uas'}).text)
                except:
                    beer_rating.append(None)
            # Scrape Brewery Name  
                try:
                    brewery_div = rows.find('div')
                    brewery_name.append(brewery_div.a.text)
                except:
                    brewery_name.append(None)
            # Scrape Brewery Location
                try:
                    brewery_location.append(rows.find('span', attrs={'class':'small'}).text.strip())
                except:
                    brewery_location.append(None)
            # Scrape Reviewer Name
                try:
                    reviewer_name.append(rows.find('span', attrs={'class':'pull-right'}).text.strip())
                except:
                    reviewer_name.append(None)
            # Scrape content
                try:
                    review_content.append(rows.find('div', attrs={'style':"color:#666;"}).text.strip())
                except:
                    review_content.append(None)
        except:
            pass

    beerz = pd.DataFrame({'beer_name': beer_name,
                         'beer_rating': beer_rating,
                        'brewery_name': brewery_name,
                         'brewery_location': brewery_location,
                        'review_content': review_content,
                        'reviewer_name': reviewer_name})
    return beerz



In [3]:
# Scraping most recent reviews
url = 'https://www.ratebeer.com/beer-ratings/0/{}/'
df_recent = ratebeer_scraper(0,100,url)

# Scraping new beers
url = 'https://www.ratebeer.com/beer-ratings/5/{}/'
df_new = ratebeer_scraper(0,10,url)

# Scraping fave beers
url = 'https://www.ratebeer.com/beer-ratings/4/{}/'
df_fave = ratebeer_scraper(0,50,url)

# Scraping rave beers
url = 'https://www.ratebeer.com/beer-ratings/1/{}/'
df_rave = ratebeer_scraper(0,50,url)

# Scraping rant beers
url = 'https://www.ratebeer.com/beer-ratings/2/{}/'
df_rant = ratebeer_scraper(0,50,url)

# Scraping anomalous beers
url = 'https://www.ratebeer.com/beer-ratings/3/{}/'
df_anom = ratebeer_scraper(0,50,url)

In [4]:
# getting rid of non-ascii characters from the reviews as it breaks csv
try:
    df_rave.review_content = df_rave.review_content.apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))
except:
    pass

In [5]:
# read current data from csv (there is a backup at all_reviews_backup.csv)
df = pd.read_csv('all_reviews.csv')

In [6]:
# concatenating all of the scrapes together with the old df
try:
    df = pd.concat([df, df_recent, df_new, df_fave, df_rave, df_rant, df_anom])
except:
    df = pd.concat([df, df_recent])
print(len(df))
df.drop_duplicates(inplace=True)
print(len(df))
df.beer_name = df.beer_name.apply(lambda x: x.strip())



30484
26371


In [7]:
df

beer_name beer_rating  \
0                             St. Peter's Ruby Red Ale         3.3   
1                     Shepherd Neame Spitfire (Bottle)         2.2   
2                                      Arbor Rocketman         3.8   
3               Shepherd Neame India Pale Ale (Bottle)         3.4   
4                         Shepherd Neame Christmas Ale         3.3   
..                                                 ...         ...   
760                                   Wild Beer Nebula         1.8   
761  Jester King Le Petit Prince Farmhouse Table Be...         5.0   
762                                 Goose Island So-Lo         5.0   
763                              Wild Beer Epic Saison         1.8   
764                              Iron City Light Mango         3.9   

             brewery_location                          brewery_name  \
0             Bungay, Suffolk                   St. Peter's Brewery   
1             Faversham, Kent                        Shepherd Neame   
2    Bristol, Gloucestershire                            Arbor Ales   
3             Faversham, Kent                        Shepherd Neame   
4             Faversham, Kent                        Shepherd Neame   
..                        ...                                   ...   
760                      -1.7                             Wild Beer   
761                      +1.6                   Jester King Brewery   
762                        +2  Goose Island Beer Company (AB-InBev)   
763                      -1.7                             Wild Beer   
764                      +1.6            Pittsburgh Brewing Company   

                                        review_content        reviewer_name  
0    Ruby color, offwhite coam. Aroma and taste: fl...            alex_leit  
1    Disappointing show from the lads at ‘Britain’s...  shivermetimber.gray  
2    Puszka. Mętne o żółtym kolorze. Piana biała,ob...              MarcinG  
3    Bottle. Amber with bronze color and white foam...            alex_leit  
4    Bottle. Aroma: English hops and malts with tof...            alex_leit  
..                                                 ...                  ...  
760  Keg at We Brought Beer, Balham.  Hazy straw wi...                Beese  
761  Light yellow liquid. Scent of hay, herbs and y...          SourScout57  
762  Genius. A delicious light beer that downplays ...          LocalLove53  
763  * Bottle 5% Farmhouse Ale.\n* Appearance is ha...        MrJamesMcEwan  
764  Nice light fruit flavored beer. Only 95 cal an...          LuvMyBeer64  

[26371 rows x 6 columns]

In [8]:
df.to_csv('all_reviews.csv',index=False)
# df = pd.read_csv('all_reviews.csv')
not_found_df = pd.read_csv("not_found.csv")

## Get more information on all of the beers - Selenium

In [9]:
from selenium import webdriver
import time

In [10]:
# extract already scraped data from previous scrapes so don't rescrape
beer_df = pd.read_csv('beer_deets.csv')

In [11]:
# compare to the unique list to create a new to-be-scraped list
new_beer = df.beer_name[(~df.beer_name.isin(beer_df.name_found))&(~df.beer_name.isin(not_found_df.not_found))].unique()
print("There are", len(new_beer), "new beers to search for!")

There are 813 new beers to search for!


In [12]:
item_names = []
desc_string = []

driver = webdriver.Chrome(executable_path='/Users/lukebetham/Downloads/chromedriver')

for beer in tqdm(new_beer):
    driver.get("https://www.ratebeer.com/search?beername={}".format(beer))
    time.sleep(2)
    item_tags = driver.find_elements_by_class_name('fg-1')

    x=1
    beer_temp = []
    for item in item_tags:        
        if x <2:
            try:
                item_names.append(item.find_element_by_class_name('fd-r').text)
                x+=1
            except:
                pass
            try:
                beer_temp.append(item.text)
            except:
                pass
    try:
        desc_string.append(beer_temp[4])
    except:
        desc_string.append(None)

driver.quit()

In [13]:
beer_deets = pd.DataFrame(desc_string, columns = ['return_string'])
beer_deets

return_string
0    Beer Tree Brew Lonely Island\n🇺🇸 IPA • 7.1%\n-...
1    Magic Rock Strongman Rye BA\n🇬🇧 BarleyWine / W...
2    Beyond the Pale Pale Ale Project\n🇨🇦 Pale Ale ...
3    Oliphant Mr. Samurai Karateguy\n🇺🇸 IPA • 5.8%\...
4    Commonwealth Inception\n🇺🇸 IIPA DIPA - Hazy / ...
..                                                 ...
808  Silly Pink Killer\n🇧🇪 Flavored - Fruit • 5.0%\...
809  Bitte Schön Puddicombe House\n🇨🇦 Pale Lager - ...
810                                               None
811  Tuque de Broue Tuque Dorée\n🇨🇦 Pale Ale - Amer...
812  Boshkung Kungaroo\n🇨🇦 IPA • 5.8%\n-\n-\n2.92\n...

[813 rows x 1 columns]

In [14]:
## cleaning the scraped data
beer_deets.dropna(inplace=True)
beer_deets = beer_deets[~beer_deets.return_string.str.startswith('Logo\n')].copy()
no_alcohol = beer_deets[~beer_deets.return_string.str.contains('%')].copy()
beer_deets = beer_deets[beer_deets.return_string.str.contains('%')].copy()


In [15]:
# sorting the scraped data into columns
beer_deets['name_found'] = beer_deets.return_string.apply(lambda x: x.split('\n')[0].strip())
beer_deets['flag'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
beer_deets['type'] = beer_deets.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
beer_deets['abv'] = beer_deets.return_string.apply(lambda x: float(x.split('\n')[1].split('•')[1].replace("%","")))
beer_deets['rb_overall_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[2])
beer_deets['rb_style_score'] = beer_deets.return_string.apply(lambda x: x.split('\n')[3])
beer_deets['rb_user_rating'] = beer_deets.return_string.apply(lambda x: x.split('\n')[4])
beer_deets['rb_rating_number'] = beer_deets.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))


In [16]:
# sorting the no alcohol measure
no_alcohol['name_found'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[0].strip())
no_alcohol['flag'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0].split(" ")[0])
no_alcohol['type'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[1].split('•')[0][2:])
no_alcohol['abv'] = no_alcohol.type.apply(lambda x: 0 if x[-1] == '0' else None)
no_alcohol['type'] = no_alcohol.type.apply(lambda x: x[:-1] if x[-1] == '0' else x)
no_alcohol['check'] = no_alcohol.return_string.apply(lambda x: len(x.split('\n')))
no_alcohol = no_alcohol[no_alcohol['check']>4].copy()
no_alcohol['rb_overall_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[2])
no_alcohol['rb_style_score'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[3])
no_alcohol['rb_user_rating'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[4])
no_alcohol['rb_rating_number'] = no_alcohol.return_string.apply(lambda x: x.split('\n')[5].replace("(","").replace(")",""))
no_alcohol.drop('check',inplace=True, axis=1)

no_alcohol

return_string  \
15   Thomas Hooker Premier\n🇺🇸 Pale Lager - Intl / ...   
16   Les Trois Mousquetaires Keller Brett Saaz\n🇨🇦 ...   
75   Sideward Audit in Progress\n🇺🇸 IIPA DIPA - Imp...   
84   S43 Juice Cannon\n🇬🇧 IPA - Hazy / NEIPA\n-\n-\...   
173  Lil Beaver Lil Hoppy One\n🇺🇸 ISA - Session IPA...   
187  Cycle Scotch Barrel-Aged Imperial Stout\n🇺🇸 St...   
207  Épitaphe MacKveiken Flower\n🇨🇦 Scotch Ale / We...   
252  Lagabière Hoppy Day\n🇨🇦 IPA - Hazy / NEIPA\n-\...   
302  Siboire Sherbière\n🇨🇦 Pilsener / Pils / Pilsne...   
316  Brasserie Générale 18e JumanSmooth : Terroir A...   
317  Bar Hop Tremelo V\n🇨🇦 Saison / Farmhouse / Gri...   
357  Joe Beef Apocalypse\n🇨🇦 Flavored - Fruit\n-\n-...   
376   Cellarmaker Mo' HBC 630\n🇺🇸 IPA\n-\n-\n3.40\n(1)   
404         Buck's Porter\n🇺🇸 Porter0\n-\n-\n3.30\n(4)   
442  Three Tigers Humble Opinion's\n🇺🇸 IIPA DIPA - ...   
509  Stoudts Believer Reserve Tripel Belgian Style ...   
536  Budweiser Zero\n🇺🇸 Low / No Alcohol Beer - Pal...   
701  Brasseurs du Monde Le Festibière de Sherbrooke...   
801  Staffelberg-Bräu Loffelder Alkoholfrei\n🇩🇪 Low...   

                                            name_found flag  \
15                               Thomas Hooker Premier   🇺🇸   
16           Les Trois Mousquetaires Keller Brett Saaz   🇨🇦   
75                          Sideward Audit in Progress   🇺🇸   
84                                    S43 Juice Cannon   🇬🇧   
173                           Lil Beaver Lil Hoppy One   🇺🇸   
187            Cycle Scotch Barrel-Aged Imperial Stout   🇺🇸   
207                         Épitaphe MacKveiken Flower   🇨🇦   
252                                Lagabière Hoppy Day   🇨🇦   
302                                  Siboire Sherbière   🇨🇦   
316  Brasserie Générale 18e JumanSmooth : Terroir A...   🇨🇦   
317                                  Bar Hop Tremelo V   🇨🇦   
357                                Joe Beef Apocalypse   🇨🇦   
376                            Cellarmaker Mo' HBC 630   🇺🇸   
404                                      Buck's Porter   🇺🇸   
442                      Three Tigers Humble Opinion's   🇺🇸   
509  Stoudts Believer Reserve Tripel Belgian Style Ale   🇺🇸   
536                                     Budweiser Zero   🇺🇸   
701  Brasseurs du Monde Le Festibière de Sherbrooke...   🇨🇦   
801             Staffelberg-Bräu Loffelder Alkoholfrei   🇩🇪   

                                   type  abv rb_overall_score rb_style_score  \
15          Pale Lager - Intl / Premium  NaN                -              -   
16                     Sour / Wild Beer  NaN                -              -   
75    IIPA DIPA - Imperial / Double IPA  NaN                -              -   
84                   IPA - Hazy / NEIPA  NaN                -              -   
173                   ISA - Session IPA  NaN                -              -   
187                    Stout - Imperial  0.0               99             93   
207              Scotch Ale / Wee Heavy  NaN                -              -   
252                  IPA - Hazy / NEIPA  NaN                -              -   
302           Pilsener / Pils / Pilsner  0.0                -              -   
316         Sour / Wild Beer - Flavored  NaN                -              -   
317       Saison / Farmhouse / Grisette  NaN                -              -   
357                    Flavored - Fruit  NaN                -              -   
376                                 IPA  NaN                -              -   
404                              Porter  0.0                -              -   
442     IIPA DIPA - Hazy / Double NEIPA  NaN                -              -   
509                              Tripel  NaN                -              -   
536        Low / No Alcohol Beer - Pale  0.0                -              -   
701                      IPA - Flavored  NaN                -              -   
801        Low / No Alcohol Beer - Pale  0.0                -              -   

    rb_user_rating

In [17]:
beer_df = pd.concat([beer_df,beer_deets,no_alcohol])

In [18]:
beer_df

return_string flag  \
0    St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...   🇬🇧   
1    Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...   🇬🇧   
2    Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...   🇬🇧   
3    Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...   🇬🇧   
4    Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...   🇬🇧   
..                                                 ...  ...   
442  Three Tigers Humble Opinion's\n🇺🇸 IIPA DIPA - ...   🇺🇸   
509  Stoudts Believer Reserve Tripel Belgian Style ...   🇺🇸   
536  Budweiser Zero\n🇺🇸 Low / No Alcohol Beer - Pal...   🇺🇸   
701  Brasseurs du Monde Le Festibière de Sherbrooke...   🇨🇦   
801  Staffelberg-Bräu Loffelder Alkoholfrei\n🇩🇪 Low...   🇩🇪   

                                 type  abv rb_overall_score rb_style_score  \
0           Bitter - Ordinary / Best   4.3               49             95   
1           Bitter - Ordinary / Best   4.5               34             59   
2                                IPA   6.0               95             97   
3                      IPA - English   6.1               46             47   
4               Strong Ale - English   7.0               45             46   
..                                ...  ...              ...            ...   
442   IIPA DIPA - Hazy / Double NEIPA  NaN                -              -   
509                            Tripel  NaN                -              -   
536      Low / No Alcohol Beer - Pale  0.0                -              -   
701                    IPA - Flavored  NaN                -              -   
801      Low / No Alcohol Beer - Pale  0.0                -              -   

    rb_user_rating rb_rating_number  \
0             3.25              415   
1             3.05             1083   
2             3.74               59   
3             3.22              420   
4             3.18              409   
..             ...              ...   
442           3.80                1   
509           3.85                2   
536           3.80                1   
701           3.40                1   
801           1.81                8   

                                            name_found  
0                             St. Peter's Ruby Red Ale  
1                     Shepherd Neame Spitfire (Bottle)  
2                                      Arbor Rocketman  
3               Shepherd Neame India Pale Ale (Bottle)  
4                         Shepherd Neame Christmas Ale  
..                                                 ...  
442                      Three Tigers Humble Opinion's  
509  Stoudts Believer Reserve Tripel Belgian Style Ale  
536                                     Budweiser Zero  
701  Brasseurs du Monde Le Festibière de Sherbrooke...  
801             Staffelberg-Bräu Loffelder Alkoholfrei  

[14764 rows x 9 columns]

In [19]:
# compare to the unique list to create a new to-be-scraped list
not_found_beer = df.beer_name[~df.beer_name.isin(beer_df.name_found)].unique()
print("There are", len(not_found_beer)-len(not_found_df), "new beers that were not found or returned errors. They will be added to the no search list.")
not_found_df = pd.concat([not_found_df,pd.DataFrame(not_found_beer,columns = ["not_found"])])
not_found_df.drop_duplicates(inplace=True)

There are 46 new beers that were not found or returned errors. They will be added to the no search list.


In [20]:
beer_df.to_csv('beer_deets.csv',index=False)
not_found_df.to_csv("not_found.csv", index=False)